In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [1]:
import os
ROOT = '/content/drive/MyDrive/Midfluencer'
INPUT = os.path.join(ROOT, "images")
OUTPUT = os.path.join(ROOT, "output")

In [6]:
MODEL = "dreamlike-art/dreamlike-photoreal-2.0"

In [3]:
!pip install -U autotrain-advanced

In [4]:
!autotrain setup --update-torch

> INFO    Installing latest xformers
> INFO    Successfully installed latest xformers
> INFO    Installing latest PyTorch
> INFO    Successfully installed latest PyTorch


In [5]:
!autotrain dreambooth --class-prompt "photo of a person" \
--model stabilityai/stable-diffusion-xl-base-1.0 \
--image-path /content/drive/MyDrive/Midfluencer/images \
--prompt "photo of LaurAI3 person" \
--resolution 1024 \
--checkpointing-steps 200 \
--batch-size 1 \
--num-steps 1000 \
--fp \
--gradient-accumulation 4 \
--lr 1e-4 \
--project-name /content/drive/MyDrive/Midfluencer/output/midfluencer_v8


> INFO    Namespace(version=False, model='stabilityai/stable-diffusion-xl-base-1.0', revision=None, tokenizer=None, image_path='/content/drive/MyDrive/Midfluencer/images', class_image_path=None, prompt='photo of LaurAI3 person', class_prompt='photo of a person', num_class_images=100, class_labels_conditioning=None, prior_preservation=None, prior_loss_weight=1.0, project_name='/content/drive/MyDrive/Midfluencer/output/midfluencer_v8', seed=42, resolution=1024, center_crop=None, train_text_encoder=None, batch_size=1, sample_batch_size=4, epochs=1, num_steps=1000, checkpointing_steps=200, resume_from_checkpoint=None, gradient_accumulation=4, gradient_checkpointing=None, lr=0.0001, scale_lr=None, scheduler='constant', warmup_steps=0, num_cycles=1, lr_power=1.0, dataloader_num_workers=0, use_8bit_adam=None, adam_beta1=0.9, adam_beta2=0.999, adam_weight_decay=0.01, adam_epsilon=1e-08, max_grad_norm=1.0, allow_tf32=None, prior_generation_precision=None, local_rank=-1, xformers=None, pre_compu

In [9]:
import pandas as pd

prompts = pd.read_excel("/content/drive/MyDrive/Midfluencer/input/20231115 Prompts vJOK.xlsx")

In [11]:
for _, row in prompts.iterrows():
  "The person" + row["Positive"]

LaurAI3, faeriete couture, styled in D&G, dynamic light, style: Photography 
LaurAI3, big smile, blue eyes, freckles, dark makeup, hyperdetailed photography, soft light, head and shoulders portrait, cover 
LaurAI3, urban scenery, 150mm, dlrs, beautiful neon soft light, vibrant details, beautiful masterwork by head of prompt engineering
LaurAI3, Street photography photo in a stylish French setting while window shopping, upper body framing, on a quiet Parisian street, with neon lighting from shop signs, shot from a high angle, on a Sony A7111 with a (bokeh effect:1.3),(in the style of Garry Winograd:1.4)
LaurAI3, photo of young woman, highlight hair, sitting outside restaurant, wearing dress, rim lighting, studio lighting, looking at the camera, dslr, ultra quality, sharp focus, tack sharp, dof, film grain, Fujifilm XT3, crystal clear, 8K UHD, highly detailed glossy eyes, high detailed skin, skin pores


In [13]:
from diffusers import DiffusionPipeline, StableDiffusionXLImg2ImgPipeline
import torch

version = "_v7"
if not os.path.exists(os.path.join(OUTPUT, f"raw{version}")):
  os.mkdir(os.path.join(OUTPUT, f"raw{version}"))
if not os.path.exists(os.path.join(OUTPUT, f"processed{version}")):
  os.mkdir(os.path.join(OUTPUT, f"processed{version}"))

pipe = DiffusionPipeline.from_pretrained(
    MODEL,
    torch_dtype=torch.float16
)
pipe.to("cuda")
pipe.load_lora_weights(os.path.join(OUTPUT, "midfluencer"+version), weight_name="pytorch_lora_weights.safetensors")

for i, row in prompts.iterrows():
  prompt = "the person " + row["Positive"]
  negative_prompt = row["Negative"]
  folder = f"raw{version}_{i}"
  if not os.path.exists(os.path.join(OUTPUT, folder)):
    os.mkdir(os.path.join(OUTPUT, folder))

  for seed in range(10):
      generator = torch.Generator("cuda").manual_seed(seed)
      image_pipe = pipe(prompt=prompt, negative_prompt=negative_prompt, generator=generator, num_inference_steps=25)
      image = image_pipe.images[0]
      image.save(os.path.join(OUTPUT, folder, f"{seed}.png"))

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

...

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]